<a href="https://colab.research.google.com/github/elidonda2-web/Alisia-7B-it-/blob/main/Chat_with_Alisia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q gradio
!pip install -q llama-cpp-python

In [ ]:
Votre_id=      #Tu dois coller devant le signe d'égalisation lien que tu as copier depuis drive.

In [ ]:
!pip install gdown --quit
!gdown Votre_id -O /kaggle/working/Alisia.Q4_K_M.gguf

In [ ]:
# **********************************************************************
# Alisis - Chat UI complete (GGUF + Streaming)
# **********************************************************************
from llama_cpp import Llama
import gradio as gr
import json
import os
import datetime

# **********************************************************************
# Paramètres du modèle
# **********************************************************************

MODEL_PATH = "Alisis.Q4.K.M.gguf"

llm = Llama(
    model_path=MODEL_PATH,
    n_ctx=4096,
    n_threads=6,
    n_gpu_layers=50,
    use_mlock=False,
    verbose=False,
)

# **********************************************************************
# Configuration par défaut
# **********************************************************************

DEFAULT_SYSTEM_PROMPT = """You are Alisis, a helpful, precise assistant created by the Alisis Research Team."""
SAVE_PATH = "chat_history.json"

# **********************************************************************
# Gestion de l'historique
# **********************************************************************

def save_chat(history):
    os.makedirs("logs", exist_ok=True)
    filename = f"logs/chat_{datetime.datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.json"
    with open(filename, "w", encoding='utf-8') as f:
        json.dump(history, f, indent=2, ensure_ascii=False)
    return filename

def load_chat():
    if os.path.exists(SAVE_PATH):
        with open(SAVE_PATH, "r", encoding='utf-8') as f:
            return json.load(f)
    return []

# **********************************************************************
# Fonction de streaming
# **********************************************************************

def stream_response(message, history, system_prompt, temperature, max_tokens):
    messages = [{"role": "system", "content": system_prompt}]

    # Ajouter l'historique
    for human, ai in history:
        messages.append({"role": "user", "content": human})
        messages.append({"role": "assistant", "content": ai})

    # Ajouter le message actuel
    messages.append({"role": "user", "content": message})

    stream = llm.create_chat_completion(
        messages=messages,
        stream=True,
        temperature=temperature,
        top_p=0.9,
        max_tokens=int(max_tokens),
    )

    partial = ""
    for chunk in stream:
        if "choices" in chunk and len(chunk["choices"]) > 0:
            delta = chunk["choices"][0]["delta"]
            if "content" in delta:
                token = delta["content"]
                partial += token
                yield partial

# **********************************************************************
# Interface utilisateur
# **********************************************************************

with gr.Blocks(theme=gr.themes.Soft(), title='Alisis - Conversational AI') as demo:
    gr.Markdown(
        """
        <h1 style='text-align: center;'>Alisis - Research Assistant</h1>
        <p style='text-align: center; color: gray;'>
        Powered by Qwen2-7B (Quantized GGUF via llama.cpp)<br>
        Created by the <b>Alisis Research Team</b>
        </p>
        """
    )

    with gr.Row():
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(height=550, label="Conversation")

            with gr.Row():
                msg = gr.Textbox(
                    placeholder='🔍 Tapez votre message ici...',
                    show_label=False,
                    lines=2,
                )
                send_btn = gr.Button("🚀 Envoyer", variant="primary")

            with gr.Row():
                clear_btn = gr.Button("🗑️ Effacer la conversation")
                save_btn = gr.Button("💾 Sauvegarder le chat")

        with gr.Column(scale=1):
            gr.Markdown("## ⚙️ Paramètres")

            system_prompt = gr.Textbox(
                value=DEFAULT_SYSTEM_PROMPT,
                label="Message système (rôle de l'assistant)",
                lines=4,
            )

            temperature = gr.Slider(
                0.1, 1.5, value=0.7, step=0.05, label="Température"
            )

            max_tokens = gr.Slider(
                128, 2048, value=512, step=64, label="Longueur maximale"
            )

            download_link = gr.File(label="📥 Télécharger le dernier historique")

    # **********************************************************************
    # Logique interactive
    # **********************************************************************

    def user_message(message, chat_history):
        return "", chat_history + [[message, None]]

    def bot_response(chat_history, system_prompt, temperature, max_tokens):
        message = chat_history[-1][0]
        response = ""
        for partial in stream_response(
            message, chat_history[:-1], system_prompt, temperature, max_tokens
        ):
            response = partial
            chat_history[-1][1] = response
            yield chat_history

    def clear_chat():
        return []

    def save_current(history):
        filename = save_chat(history)
        return gr.File(value=filename)

    # **********************************************************************
    # Liaison des actions
    # **********************************************************************

    msg.submit(
        user_message,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    ).then(
        bot_response,
        [chatbot, system_prompt, temperature, max_tokens],
        chatbot
    )

    send_btn.click(
        user_message,
        [msg, chatbot],
        [msg, chatbot],
        queue=False
    ).then(
        bot_response,
        [chatbot, system_prompt, temperature, max_tokens],
        chatbot
    )

    clear_btn.click(clear_chat, None, chatbot, queue=False)
    save_btn.click(save_current, chatbot, download_link)

    # **********************************************************************
    # Lancement
    # **********************************************************************

    if __name__ == '__main__':
        demo.queue()
        demo.launch(server_name='0.0.0.0', share=True)